In [ ]:
from tensorflow import keras
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import os
import numpy as np
import pandas as pd

import sys

from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

import matplotlib.pyplot as plt

In [ ]:
# IMPORT EXPERIMENTS DATA
results_folder = os.getcwd()
csv_details = "EXPERIMENTS.csv"

exp_details = pd.read_csv(results_folder+"/"+csv_details)

content = os.listdir(results_folder)
exp_dirs = [d for d in content if os.path.isdir(d)][1:]
exp_dirs = sorted(exp_dirs, key=lambda x: int(x.split("_")[0]))

folder_plots = "plots/delta/"

In [ ]:
# MODELS
root = "C:/0_thesis/3_experiment/"
models_rgb = []
models_rgb.append(os.path.join(root, "models/rgb_no_alpha/model_4"))
models_rgb.append(os.path.join(root, "models/rgb_alpha05/model_4"))
models_rgb.append(os.path.join(root, "models/rgb_alpha1/model_4"))

models_gray = []
models_gray.append(os.path.join(root, "models/gray_no_alpha/model_4"))
models_gray.append(os.path.join(root, "models/gray_alpha05/model_4"))
models_gray.append(os.path.join(root, "models/gray_alpha1/model_4"))

models = models_rgb + models_gray
models_names = []
for model_path in models:
    models_names.append(model_path.split("/")[-2])

# PREDICT ON ALL MODELS

In [ ]:
def predict(model_path, images_path, colormode):
    # Load model
    model = keras.models.load_model(model_path)

    # Predict on each image
    img_size = 256
    filenames = os.listdir(images_path)

    ages = []
    genders = []

    for filename in filenames:
        if colormode in filename:
            filepath = images_path+"/"+filename
        
            img = tf.keras.utils.load_img(filepath, target_size = (img_size, img_size), color_mode=colormode)
            img = tf.keras.utils.img_to_array(img)
            img = img * (1./255)
            img = tf.expand_dims(img, axis = 0)
            prediction = model.predict(img)
            prediction = np.round(prediction)
            age_pred = int(prediction[0])
            ages.append(age_pred)
            gender_pred = "male" if prediction[1] == 0 else "female"
            genders.append(gender_pred)

    # Analysis
    age_avg = sum(ages)/len(ages)
    #print("Max age: ", max(ages))
    #print("Min age: ", min(ages))
    #print("Average: ", age_avg)
    #print("Most gender: ", max(set(genders), key=genders.count))

    final_age = np.round(age_avg)
    final_gender = max(set(genders), key=genders.count)

    return final_age, final_gender

In [ ]:
# DATAFRAMES: 1 WITH PREDICTED AGE, 1 WITH PREDICTED GENDER 
df_age = pd.DataFrame(exp_dirs, columns = ['DIRNAME'] )
df_age['ID'] = exp_details['ID']
df_age['BEHAVIOR'] = exp_details['BEHAVIOR']
df_age['AGE'] = exp_details['AGE']


df_gender = pd.DataFrame(exp_dirs, columns = ['DIRNAME'] )
df_gender['ID'] = exp_details['ID']
df_gender['BEHAVIOR'] = exp_details['BEHAVIOR']
df_gender['GENDER'] = exp_details['GENDER']

In [ ]:
# PREDICT AND SAVE IN DATAFRAMES
for experiment in exp_dirs:
    print("EXPERIMENT: ", experiment)
    images_path = results_folder+"/"+experiment+"/photos/"
    row = df_age[df_age['DIRNAME'] == experiment].index
    
    for model_path in models_rgb:
        age, gender = predict(model_path, images_path, colormode="rgb")
        model = model_path.split("/")[-2]
        df_age.loc[row, model] = age
        df_gender.loc[row, model] = gender

    for model_path in models_gray:
        age, gender = predict(model_path, images_path, colormode="grayscale")
        model = model_path.split("/")[-2]
        df_age.loc[row, model] = age
        df_gender.loc[row, model] = gender

In [ ]:
df_age.to_csv("age_prediction.csv", index=False)
df_gender.to_csv("gender_prediction.csv", index=False)

## BALANCED MODELS

In [ ]:
'''
temp_root = "C:/0_thesis/2_model/TESTING/BALANCE/"
models_rgb = []
models_rgb.append(os.path.join(temp_root, "19fold/model_4"))
models_rgb.append(os.path.join(temp_root, "20netto/model_4"))
models_rgb.append(os.path.join(temp_root, "20pepper05/model_4"))
models_gray = []
models_gray.append(os.path.join(temp_root, "19foldgray/model_4"))
models_gray.append(os.path.join(temp_root, "20nettogray/model_4"))
models_gray.append(os.path.join(temp_root, "20pepper05gray/model_4"))

# DATAFRAMES: 1 WITH PREDICTED AGE, 1 WITH PREDICTED GENDER 
df_balance_age = pd.DataFrame(exp_dirs, columns = ['DIRNAME'] )
df_balance_age['ID'] = exp_details['ID']
df_balance_age['BEHAVIOR'] = exp_details['BEHAVIOR']
df_balance_age['AGE'] = exp_details['AGE']

df_balance_gender = pd.DataFrame(exp_dirs, columns = ['DIRNAME'] )
df_balance_gender['ID'] = exp_details['ID']
df_balance_gender['BEHAVIOR'] = exp_details['BEHAVIOR']
df_balance_gender['GENDER'] = exp_details['GENDER']

# PREDICT AND SAVE IN DATAFRAMES
for experiment in exp_dirs:
    print("EXPERIMENT: ", experiment)
    images_path = results_folder+"/"+experiment+"/photos/"
    row = df_balance_age[df_balance_age['DIRNAME'] == experiment].index
    
    for model_path in models_rgb:
        age, gender = predict(model_path, images_path, colormode="rgb")
        model = model_path.split("/")[-2]
        df_balance_age.loc[row, model] = age
        df_balance_gender.loc[row, model] = gender

    for model_path in models_gray:
        age, gender = predict(model_path, images_path, colormode="grayscale")
        model = model_path.split("/")[-2]
        df_balance_age.loc[row, model] = age
        df_balance_gender.loc[row, model] = gender
        
df_balance_age.to_csv("balance_age_prediction.csv", index=False)
df_balance_gender.to_csv("belance_gender_prediction.csv", index=False)
'''

# ANALYSIS

In [ ]:
age_ranges = {'experiment': [range(0, 18), range(18, 30), range(30, 40), range(40, 50), 
                             range(50, 60), range(60, 70), range(70, 117)],
              'paper2019': [range(0, 16), range(16, 30), range(30, 50), range(50, 117)],
              'generations': [range(0, 11), range(11, 27), range(27, 43), range(43, 59), range(59, 78), range(78, 95)],
              'paper_bari': [range(0, 26), range(26, 41), range(41, 61), range(61, 120)]}

def get_group_from_age(age, age_groups):
    for i, r in enumerate(age_groups):
        if age in r:
            return i
    return None

def get_groups_from(ages):
    age_groups = dict.fromkeys(age_ranges)
    for group_name in age_ranges.keys():
        age_groups[group_name] = []
        for age in ages:
            age_groups[group_name].append(get_group_from_age(age, age_ranges[group_name]))
    return age_groups

## Accuracy, MAE, precision, recall, F1-score

In [ ]:
df_age = pd.read_csv("age_prediction.csv", index_col=0)
df_gender = pd.read_csv("gender_prediction.csv", index_col=0)
#df_age = pd.read_csv("balance_age_prediction.csv")
#df_gender = pd.read_csv("belance_gender_prediction.csv")

In [ ]:
# ACCURACY ON GENDER
gender_mapper = {'male': 0, 'female': 1}
df_gender = df_gender.replace({"GENDER": gender_mapper})

print("ACCURACY -- GENDER")

models = models_rgb + models_gray

for model_path in models:
    model = model_path.split("/")[-2]
    df_gender = df_gender.replace({model: gender_mapper})
    accuracy = accuracy_score(df_gender["GENDER"], df_gender[model])
    print(model+": ", accuracy)

In [ ]:
# true age groups
age_groups = get_groups_from(df_age["AGE"].tolist())

In [ ]:
print("ACCURACY -- AGE GROUP")

models = models_rgb + models_gray

# for every model
for model_path in models:
    model = model_path.split("/")[-2]
    print("------"+model+"-------")
    # for every age_ranges
    pred_age_groups = get_groups_from(df_age[model].tolist())
    for group_name in age_ranges.keys():
        accuracy = accuracy_score(age_groups[group_name], pred_age_groups[group_name])
        print(group_name+":",np.round(accuracy,2))

In [ ]:
from sklearn.metrics import mean_absolute_error

print("MAE -- AGE")

models = models_rgb + models_gray

mae = []
for model_path in models:
    model = model_path.split("/")[-2]
    mae.append(mean_absolute_error(y_true=df_age["AGE"].tolist(), y_pred=df_age[model].tolist()))
    print(model+": ", mae[-1])

# Indexes of wrong predictions

In [ ]:
pred_age_groups = get_groups_from(df_age["gray_no_alpha"].tolist())['experiment']
age_groups = get_groups_from(df_age["AGE"].tolist())['experiment']
genders = df_gender['GENDER'].tolist()
pred_genders = df_gender["gray_no_alpha"].tolist()
tot_experiments = int(len(genders)/2)
tot_predictions = tot_experiments*2

In [ ]:
import matplotlib.pyplot as plt

offset_youngest = {}
offset_oldest = {}
marker_size = 5
color_line = "gray"

for model in models_names:
    diff_age = df_age[model] - df_age["AGE"]
    predictions = df_age['ID']
    
    offset_youngest[model] = min(diff_age)
    offset_oldest[model] = max(diff_age)
       
    fig, ax = plt.subplots(1,1,figsize=(16, 12))
    
    # points with error between -5 and -2.5 or 2.5 and 5
    plt.plot(predictions, diff_age, 'yo', markersize=marker_size)

    # error less than -5 or more than 5: red points
    plt.axhline(y = 5, color = color_line)
    plt.axhline(y = -5, color = color_line)
    diff_age_red = []
    exps_red = []
    for i in range(len(diff_age)):
        if diff_age[i] < -5 or diff_age[i] > 5:
            diff_age_red.append(diff_age[i])
            exps_red.append(predictions[i])
    plt.plot(exps_red, diff_age_red, 'ro', markersize=marker_size)

    # error between -2.5 and 2.5: green points
    plt.axhline(y = 2.5, color = color_line)
    plt.axhline(y = -2.5, color = color_line)
    diff_age_green = []
    exps_green = []
    for i in range(len(diff_age)):
        if diff_age[i] > -2.5 and diff_age[i] < 2.5:
            diff_age_green.append(diff_age[i])
            exps_green.append(predictions[i])
    plt.plot(exps_green, diff_age_green, 'go', markersize=marker_size)

    plt.xticks(np.arange(min(exps), max(exps)+1, 1))
    plt.yticks(np.arange(-45, 52.5, 2.5))
    plt.grid(axis='x')
    plt.xlabel('Subjects')
    plt.ylabel('Years of difference between the predicted and the actual age')
    plt.title(model)
    plt.savefig(folder_plots+'/'+model+".jpg")
    plt.show()

In [ ]:
for model in models_names:
    print(model,"\t",offset_youngest[model],"\t",offset_oldest[model])

## GODSPEED

Explicit: true; false age-group/younger; false age-group/older or wrong gender

Implicit: true; false age-group/younger; false age-group/older or wrong gender

In [ ]:
# EXPLICIT
id_true_pred = []
id_younger_pred = []
id_older_pred = [] # or wrong gender
tot_experiments = len(age_groups)

for i in range(tot_experiments):
    if df_age.iloc[i].BEHAVIOR == 'explicit':
        exp_id = df_age.iloc[i].ID
        if genders[i] != pred_genders[i] or pred_age_groups[i] > age_groups[i]:
            id_older_pred.append(exp_id)
        elif pred_age_groups[i] < age_groups[i]:
            id_younger_pred.append(exp_id)
        else:
            id_true_pred.append(exp_id)
            
print("All EXPLICIT experiments are considered: ", (len(id_older_pred)+len(id_younger_pred)+len(id_true_pred))==(tot_experiments/2))

print("Id experiments with true predictions: ", id_true_pred)
print("Id experiments with younger predictions: ", id_younger_pred)
print("Id experiments with older predictions or wrong gender: ", id_older_pred)

In [ ]:
# IMPLICIT
id_true_pred = []
id_younger_pred = []
id_older_pred = [] # or wrong gender
tot_experiments = len(age_groups)

for i in range(tot_experiments):
    if df_age.iloc[i].BEHAVIOR == 'implicit':
        exp_id = df_age.iloc[i].ID
        if genders[i] != pred_genders[i] or pred_age_groups[i] > age_groups[i]:
            id_older_pred.append(exp_id)
        elif pred_age_groups[i] < age_groups[i]:
            id_younger_pred.append(exp_id)
        else:
            id_true_pred.append(exp_id)
            
print("All IMPLICIT experiments are considered: ", (len(id_older_pred)+len(id_younger_pred)+len(id_true_pred))==(tot_experiments/2))

print("Id experiments with true predictions: ", id_true_pred)
print("Id experiments with younger predictions: ", id_younger_pred)
print("Id experiments with older predictions or wrong gender: ", id_older_pred)

## TRUST

Considering the order of submission of the types of questionnaires.

## E+I

In [ ]:
# EXPLICIT first behavior
id_true_pred = []
id_younger_pred = []
id_older_pred = [] # or wrong gender
tot_experiments = len(age_groups)

for i in range(tot_experiments):
    if '_1_e' in df_age.iloc[i].DIRNAME:
        exp_id = df_age.iloc[i].ID
        if genders[i] != pred_genders[i] or pred_age_groups[i] > age_groups[i]:
            id_older_pred.append(exp_id)
        elif pred_age_groups[i] < age_groups[i]:
            id_younger_pred.append(exp_id)
        else:
            id_true_pred.append(exp_id)
            
print("Id experiments with true predictions: ", id_true_pred)
print("Id experiments with younger predictions: ", id_younger_pred)
print("Id experiments with older predictions or wrong gender: ", id_older_pred)

In [ ]:
# IMPLICIT second behavior
id_true_pred_2 = []
id_younger_pred_2 = []
id_older_pred_2 = [] # or wrong gender
tot_experiments = len(age_groups)

for i in range(tot_experiments):
    if '_2_i' in df_age.iloc[i].DIRNAME:
        exp_id = df_age.iloc[i].ID
        if genders[i] != pred_genders[i] or pred_age_groups[i] > age_groups[i]:
            id_older_pred_2.append(exp_id)
        elif pred_age_groups[i] < age_groups[i]:
            id_younger_pred_2.append(exp_id)
        else:
            id_true_pred_2.append(exp_id)

print("Id experiments with true predictions:", id_true_pred_2)
print("Id experiments with younger predictions:", id_younger_pred_2)
print("Id experiments with older predictions or wrong gender:", id_older_pred_2)

## I+E

In [ ]:
# IMPLICIT first behavior
id_true_pred = []
id_younger_pred = []
id_older_pred = [] # or wrong gender
tot_experiments = len(age_groups)

for i in range(tot_experiments):
    if '_1_i' in df_age.iloc[i].DIRNAME:
        exp_id = df_age.iloc[i].ID
        if genders[i] != pred_genders[i] or pred_age_groups[i] > age_groups[i]:
            id_older_pred.append(exp_id)
        elif pred_age_groups[i] < age_groups[i]:
            id_younger_pred.append(exp_id)
        else:
            id_true_pred.append(exp_id)

print("Id experiments with true predictions:", id_true_pred)
print("Id experiments with younger predictions:", id_younger_pred)
print("Id experiments with older predictions or wrong gender:", id_older_pred)

In [ ]:
# EXPLICIT second behavior
id_true_pred_2 = []
id_younger_pred_2 = []
id_older_pred_2 = [] # or wrong gender
tot_experiments = len(age_groups)

for i in range(tot_experiments):
    if '_2_e' in df_age.iloc[i].DIRNAME:
        exp_id = df_age.iloc[i].ID
        if genders[i] != pred_genders[i] or pred_age_groups[i] > age_groups[i]:
            id_older_pred_2.append(exp_id)
        elif pred_age_groups[i] < age_groups[i]:
            id_younger_pred_2.append(exp_id)
        else:
            id_true_pred_2.append(exp_id)

print("Id experiments with true predictions:", id_true_pred_2)
print("Id experiments with younger predictions:", id_younger_pred_2)
print("Id experiments with older predictions or wrong gender:", id_older_pred_2)

In [ ]:
print("All IMPLICIT experiments are considered:", (len(id_older_pred)+len(id_younger_pred)+len(id_true_pred)+len(id_older_pred_2)+len(id_younger_pred_2)+len(id_true_pred_2))==(tot_experiments/2))

In [ ]:
print("All IMPLICIT experiments are considered:", (len(id_older_pred)+len(id_younger_pred)+len(id_true_pred)+len(id_older_pred_2)+len(id_younger_pred_2)+len(id_true_pred_2))==(tot_experiments/2))

## Prediction stability

In [ ]:
# Where age group or gender was predicted wrong in the experiment session
pred_age_groups = get_groups_from(df_age["gray_no_alpha"].tolist())['experiment']
age_groups = get_groups_from(df_age["AGE"].tolist())['experiment']
genders = df_gender['GENDER'].tolist()
pred_genders = df_gender["gray_no_alpha"].tolist()
tot_experiments = int(len(genders)/2)
tot_predictions = tot_experiments*2

In [ ]:
# STABILITY OF ALL PREDICTIONS:
# if predictions of the two parts are equals -> stable
# not considering if predictions are true or wrong 
# so wrong-wrong is stable 
id_diff_pred_age = []
id_diff_pred_gender = []
id_diff_pred = []
for i in range(0, len(age_groups)-1, 2):
    if pred_genders[i] != pred_genders[i+1] or pred_age_groups[i] != pred_age_groups[i+1]:
        id_diff_pred.append(df_gender.iloc[i].ID)
        if pred_genders[i] != pred_genders[i+1]:
            id_diff_pred_gender.append(df_gender.iloc[i].ID)
        if pred_age_groups[i] != pred_age_groups[i+1]:
            id_diff_pred_age.append(df_gender.iloc[i].ID)

In [ ]:
print("Number of predictions different in the two parts of experiment:", len(id_diff_pred), "on", tot_experiments)
print("Number of AGE GROUP predictions different in the two parts of experiment:", len(id_diff_pred_age), "on", tot_experiments)
print("Number of GENDER predictions different in the two parts of experiment:", len(id_diff_pred_gender), "on", tot_experiments)

In [ ]:
# return array of indexes of wrong prediction in only one of the two parts
def clear_id_wrong_pred(id_wrong_pred):
    # get indexes repeated once + numbers of wrong prediction in the same id session
    # = 1 if it was wrong in only one part, 2 if it was wrong in both 
    unique_id_wrong_pred, frequency = np.unique(id_wrong_pred, return_counts = True)

    # keep only if it was wrong in one:
    # this means that prediction was not stable
    indices = np.where(frequency==2)
    return np.delete(unique_id_wrong_pred, indices)


# STABILITY OF WRONG PREDICTIONS
# if predictions of the two parts are: wrong-wrong -> stable
id_wrong_pred = []
id_wrong_pred_gender = []
id_wrong_pred_age = []

# add index of wrong predictions
for i in range(0, len(age_groups)-1):
    # if first prediction is wrong
    if genders[i] != pred_genders[i] or age_groups[i] != pred_age_groups[i]:
        id_wrong_pred.append(df_gender.iloc[i].ID)
        if genders[i] != pred_genders[i]:
            id_wrong_pred_gender.append(df_gender.iloc[i].ID)
        if age_groups[i] != pred_age_groups[i]:
            id_wrong_pred_age.append(df_gender.iloc[i].ID)

# keep index only if one of the two predictions is wrong
# so the other is true and that experiment is not stable
id_single_wrong_pred = clear_id_wrong_pred(id_wrong_pred)
id_single_wrong_pred_age = clear_id_wrong_pred(id_wrong_pred_age)
id_single_wrong_pred_gender = clear_id_wrong_pred(id_wrong_pred_gender)

In [ ]:
print("Number of predictions in which only one of the two is wrong:", len(id_single_wrong_pred), "on", tot_experiments)
print("Number of AGE GROUP predictions in which only one of the two is wrong:", len(id_single_wrong_pred_age), "on", tot_experiments)
print("Number of GENDER predictions in which only one of the two is wrong:", len(id_single_wrong_pred_gender), "on", tot_experiments)

In [ ]:
print(id_diff_pred_age)
print(id_single_wrong_pred_age)
# EXP 2: real_age_group = 1, pred_age_group_1 = 2, pred_age_group_2 = 3
# both wrong so not in id_wrong_pred_age
# but different so in id_diff_pred_age 

In [ ]:
print("% of AGE GROUP predictions equals in the two parts of experiment:", (1-np.round(len(id_diff_pred_age)/tot_experiments,2))*100)
print("% of GENDER predictions equals in the two parts of experiment:", (1-np.round(len(id_diff_pred_gender)/tot_experiments,2))*100)